<a href="https://colab.research.google.com/github/Arjun9271/Hands_on_llms/blob/main/chapter_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Classification

- using llms for text-classification.

- it can be achieved by both representative models like(BERT ,RoBerTa)

- and the generative models(gpt-3)

In [1]:
!pip install datasets transformers sentence-transformers openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# Data

In [2]:
from datasets import load_dataset

# load data

data = load_dataset("rotten_tomatoes")
data

README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [6]:
data["train"][0]

{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
 'label': 1}

In [8]:
data["train"][-1]

{'text': 'things really get weird , though not particularly scary : the movie is all portent and no content .',
 'label': 0}

# Using a Task-specific Model

In [9]:
from transformers import pipeline

model_path = "cardiffnlp/twitter-roberta-base-sentiment"

# load model into pipeline

pipe = pipeline(

    model = model_path,
    tokenizer = model_path,
    return_all_scores = True,
    device = "cuda:0"

)

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


# passing the data to the model pipe line:

In [10]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

# run the inference

y_pred = []
for output in tqdm(pipe(KeyDataset(data["test"],"text")),total = len(data["test"])):
                negative_score = output[0]["score"]
                positive_score = output[2]["score"]
                assignment = np.argmax([negative_score,positive_score])
                y_pred.append(assignment)



Disabling tokenizer parallelism, we're using DataLoader multithreading already
100%|██████████| 1066/1066 [00:21<00:00, 50.59it/s] 


In [17]:
y_pred[50]

0

In [18]:
from sklearn.metrics import classification_report

def evaluate_performance(y_true, y_pred):
         performance = classification_report(
          y_true,y_pred,
           target_names = ["Negative Review","Positive Review"]
           )
         print(performance)



In [19]:
evaluate_performance(data["test"]["label"],y_pred)

                 precision    recall  f1-score   support

Negative Review       0.76      0.83      0.80       533
Positive Review       0.81      0.74      0.78       533

       accuracy                           0.79      1066
      macro avg       0.79      0.79      0.79      1066
   weighted avg       0.79      0.79      0.79      1066

